# Task 1: Census Data Enrichment
- __Date__: June 12, 2020
- __Description__: This task is to retrieve relevant census data, officially published by statcan.gc.ca, according to users' geospatial data, i.e. latitude and longitude.

## Part 1
- __Description__: Part 1 is to find DAUIDs where a list of latitude and longitude belongs to.

### 1.1. Import necessary libraries and dataset

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import *
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
df_LatLong = pd.read_csv('long_lat_test2.csv', header=0)

### 1.2. Function to return a list of DAUIDs where list of latitudes and longitudes belong to

In [6]:
# first function, getting DAUIDs

def findDAUID(long_list, lat_list):
    
# empty arrays to store the final DAUID ouputs    
    DAUID_list = []
    PRUID_list = []
    
# shapefile directory. Reading data
     
    provinceBound = gpd.read_file("./mydata/shapefileProvPoly")
    polygonBound = gpd.read_file("./mydata/shapefileDMPoly")

# loop through the user input coordinate, search through the boundary
# shapefile to find which province the input coordinates belong. Then 
# output the unique PRUID value for that province.
    for c in range(len(lat_list)):
        
        point = Point(long_list[c], lat_list[c])
        dm1 = provinceBound.intersects(point)
        for i in range(len(dm1)):
            if dm1[i] == True:
                provID = provinceBound.PRUID[i]
                PRUID_list.append(provID)
                
# the PRUID values are used to limit the search scope of the polygons shapefile 
# to a particular province. The polygons within such province are then compared to
# user coordinate points using intersect. The list of DAUIDS for the final 
# polygons becomes the output.
                polyProv = polygonBound[polygonBound['PRUID'] == provID]
                polyProv = polyProv.reset_index()

                dm2 = polyProv.intersects(point)
                for j in range(len(dm2)):
                    if dm2[j] == True:
                        poly = polyProv.DAUID[j]
                        DAUID_list.append(poly)
            
    return list(map(int, DAUID_list)), list(map(int, list(dict.fromkeys(PRUID_list))))

In [7]:
lat_list = df_LatLong.Lat.to_list()
long_list = df_LatLong.Long.to_list()

In [8]:
DAUID_list, PRUID_list = findDAUID(long_list, lat_list)

In [9]:
PRUID_list

[61, 60]

In [10]:
fileList=[]
for i in PRUID_list:
    if i == (10 or 11 or 12 or 13):
        fileList.append('ATLANTIC')
    elif i == 24:
        fileList.append('QC')
    elif i == 35:
        fileList.append('ON')
    elif i == (46 or 47 or 48):
        fileList.append('PRAIRIES')
    elif i == 59:
        fileList.append('BC')
    elif i == (60 or 61 or 62):
        fileList.append('TERRITORIES')
list(dict.fromkeys(fileList))

['TERRITORIES']

In [15]:
region = list(dict.fromkeys(fileList))[0]
region

'TERRITORIES'

In [11]:
DAUID_list

[61060072, 60010175, 60010195, 60010189]

## Part 2
- __Description__: Part 2 is to eventually get access to data with inputs of DAUID and keyword. To do that, the following steps were carried out.<br>
    01. Import dataset<br>
    02. Organize dataset: drop unnecessary columns and rename some long-name columns into short ones.<br>
    03. Split one dataframe into multiples by primary attribute. Multiple dataframes are saved in a dictionary.
    04. Create a function that returns relevant dataframe with inputs of DAUID and keyword

### 2.1. Import necessary libraries and dataset

In [12]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [16]:
if region == 'ATLANTIC':
    df = pd.read_pickle('ATLANTIC.pkl')
elif region == 'QC':
    df = pd.read_pickle('QUEBEC.pkl')
elif region == 'ON':
    df = pd.read_pickle('ONTARIO.pkl')
elif region == 'PRAIRIES':
    df = pd.read_pickle('PRAIRIES.pkl')
elif region == 'BC':
    df = pd.read_pickle('BRITISH_COLUMBIA.pkl')
elif region == 'TERRITORIES':
    df = pd.read_pickle('TERRITORIES.pkl')

# ATLANTIC: NL, NB, NS
# BC: BC
# ON: ON
# PRAIRIES: AB, SK, MB
# QC: QC
# TERRITORIES: NT, YT
# We'd like to import entire Canada dataset to df.

In [17]:
df.head()

,GEO_CODE (POR),GEO_NAME,DIM: Profile of Dissemination Areas (2247),Member ID: Profile of Dissemination Areas (2247),Notes: Profile of Dissemination Areas (2247),Dim: Sex (3): Member ID: [1]: Total - Sex,Dim: Sex (3): Member ID: [2]: Male,Dim: Sex (3): Member ID: [3]: Female
0,1,Canada,"Population, 2016",1,1.0,35151728,...,...
1,1,Canada,"Population, 2011",2,2.0,33476688,...,...
2,1,Canada,"Population percentage change, 2011 to 2016",3,NaN,5.0,...,...
3,1,Canada,Total private dwellings,4,3.0,15412443,...,...
4,1,Canada,Private dwellings occupied by usual residents,5,4.0,14072079,...,...


In [18]:
df.rename(columns={'DIM: Profile of Dissemination Areas (2247)':'Attribute','Member ID: Profile of Dissemination Areas (2247)':'Att_id','Dim: Sex (3): Member ID: [1]: Total - Sex':'Total Sex','Dim: Sex (3): Member ID: [2]: Male':'Male','Dim: Sex (3): Member ID: [3]: Female':'Female', 'Notes: Profile of Dissemination Areas (2247)':'No','GEO_CODE (POR)':'DAUID','GEO_NAME':'Area'}, inplace=True)

In [19]:
df.head()

,DAUID,Area,Attribute,Att_id,No,Total Sex,Male,Female
0,1,Canada,"Population, 2016",1,1.0,35151728,...,...
1,1,Canada,"Population, 2011",2,2.0,33476688,...,...
2,1,Canada,"Population percentage change, 2011 to 2016",3,NaN,5.0,...,...
3,1,Canada,Total private dwellings,4,3.0,15412443,...,...
4,1,Canada,Private dwellings occupied by usual residents,5,4.0,14072079,...,...


### 2.2. Organize dataset

__2.2.1. Attribute Dataframe__

In [20]:
df_att = df[["DAUID","Att_id","Attribute"]]
df_att = df_att.head(int(df_att['Att_id'].max()))

In [21]:
df_att

,DAUID,Att_id,Attribute
0,1,1,"Population, 2016"
1,1,2,"Population, 2011"
2,1,3,"Population percentage change, 2011 to 2016"
3,1,4,Total private dwellings
4,1,5,Private dwellings occupied by usual residents
...,...,...,...
2242,1,2243,Migrants
2243,1,2244,Internal migrants
2244,1,2245,Intraprovincial migrants
2245,1,2246,Interprovincial migrants


### 2.3. Create database with splitting dataframe by attribute

In [30]:
seriesObj = df.apply(lambda x: True if x['No'] > 0 else False , axis=1)
 
# Count number of True in series
numOfRows = len(df[seriesObj == True].index)
 
print('Number of Rows in dataframe in which No > 0 : ', numOfRows)
print('The number of dataframes is',numOfRows - 1)

Number of Rows in dataframe in which No > 0 :  75825
The number of dataframes is 75824


__2.3.1. Check the row index of column 'No' having NaN value__

In [22]:
rows_NaN = df[['No']].isnull().any(axis=1)
No_NaN_List = df[rows_NaN].index.tolist()

In [31]:
#No_NaN_List

__2.3.2. Check the row index of column 'No' having non-NaN value__

In [24]:
rows_NotNaN = df[['No']].notnull().any(axis=1)
No_NotNaN_List = df[rows_NotNaN].index.tolist()

In [25]:
len(No_NotNaN_List)

75825

__As the row index having numeric(non-NaN) value is where primary attribute is, we can obtain a list of primary attribute ID.__

In [27]:
Primary_Att_id_List = []
for i in No_NotNaN_List:
    Primary_Att_id_List.append(df.iloc[[i]]["Att_id"].values[0])

In [39]:
Primary_Att_id_List[0:30]

[1,
 2,
 4,
 5,
 8,
 40,
 41,
 44,
 50,
 51,
 59,
 68,
 92,
 94,
 95,
 100,
 105,
 110,
 111,
 112,
 381,
 650,
 661,
 672,
 683,
 684,
 685,
 686,
 687,
 688]

__2.3.3 Make a dictionary of dataframes__

In [32]:
dict_of_df = {}
for i in range(numOfRows-1):
    dict_of_df["df_{}".format(i)] = df.iloc[No_NotNaN_List[i]:No_NotNaN_List[i+1],:]
    if i == numOfRows - 2:
        dict_of_df["df_{}".format(i)] = df.iloc[No_NotNaN_List[i]:,:]
    #dict_of_df["df_{}".format(i)].to_csv('{}.csv'.format(dict_of_df["df_{}".format(i)]["Data"][No_NotNaN_List[i]]))

In [96]:
dict_of_df["df_4"]

,DAUID,Area,Attribute,Att_id,No,Total Sex,Male,Female
7,1,Canada,Total - Age groups and average age of the popu...,8.0,5.0,35151730,17264200,17887530
8,1,Canada,0 to 14 years,9.0,NaN,5839570,2992925,2846645
9,1,Canada,0 to 4 years,10.0,NaN,1898790,973030,925755
10,1,Canada,5 to 9 years,11.0,NaN,2018130,1034685,983445
11,1,Canada,10 to 14 years,12.0,NaN,1922645,985200,937445
12,1,Canada,15 to 64 years,13.0,NaN,23376530,11576130,11800400
13,1,Canada,15 to 19 years,14.0,NaN,2026155,1039215,986945
14,1,Canada,20 to 24 years,15.0,NaN,2242690,1144495,1098205
15,1,Canada,25 to 29 years,16.0,NaN,2285990,1144475,1141515
16,1,Canada,30 to 34 years,17.0,NaN,2329395,1148295,1181105


### 2.4. Create a function that returns relevant dataframe with inputs of Geocode(DAUID) and keyword

__2.4.1. Function that returns entire relevant data frames matching with a list of DAUIDs and list of keywords__

In [33]:
def FindEntire(list_DAUID, list_keyword):
    try:
        dfList=[]
        for i in range(numOfRows-1):
            if dict_of_df["df_{}".format(i)]["DAUID"].unique()[0] in list_DAUID and any(x in pd.Series(dict_of_df["df_{}".format(i)]["Attribute"]).to_string().lower() for x in list_keyword):
                a = "df_{}".format(i)
                dfList.append(a)
        return dfList
    finally:
        frames=[]
        for i in range(len(dfList)):
            a = dict_of_df["{}".format(dfList[i])]
            frames.append(a)
        result = pd.concat(frames)
        result = result[["DAUID","Att_id","Total Sex"]].set_index(["DAUID"])
        return result

In [34]:
FindEntire(DAUID_list,["income","commut"])

,Att_id,Total Sex
DAUID,,
60010189,661,300
60010189,662,290
60010189,663,41088
60010189,664,290
60010189,665,36779
...,...,...
60010195,1938,315
60010195,1939,60
60010195,1940,20


__2.4.2. Function that returns specific attribute(data) matching with a list of DAUIDs and a list of attribute IDs.__

In [35]:
def FindDF(list_DAUID, list_AttID):
    df2 = pd.DataFrame(columns=df.columns)
    result2 = pd.DataFrame(columns=df.columns)
    # Separate list_AttID into two: Primary(Pr) and Secondary(Sc)
    Pr_List=[]
    Sc_List=[]
    for i in list_AttID:
        if i in Primary_Att_id_List:
            Pr_List.append(i)
        else:
            Sc_List.append(i)
        
    # Create df2 for final result
    for i in range(df.shape[0]):
        if df["DAUID"][i] in list_DAUID and df["Att_id"][i] in list_AttID:
            df2 = df2.append(df.iloc[[i]])
    df2 = df2[["DAUID","Att_id","Total Sex"]].set_index(["DAUID"])

    # See if any of elements in list_AttID includes primary attribute id.
    # If it does, ask user if they would like to see detail table with sub-attributes for primary attribute chosen.
    # Depending on user's answer(Y/N), it shows either df2 or detail table.
    # If not, show df2 (It won't ask any question.)
    if any(x in list_AttID for x in Primary_Att_id_List):
        ans = input("Primary attribute ID is included in your list provided. \nWould you like to see detail table with sub-attributes? [Y/N] ")
        
        if ans == "Y":
            dfList=[]
            for i in range(numOfRows-1):
                if dict_of_df["df_{}".format(i)]["DAUID"].unique()[0] in list_DAUID and any(x in Pr_List for x in dict_of_df["df_{}".format(i)]["Att_id"]):
                    a = "df_{}".format(i)
                    dfList.append(a)
        
            frames=[]
            for i in range(len(dfList)):
                a = dict_of_df["{}".format(dfList[i])]
                frames.append(a)
            result1 = pd.concat(frames)
            
            for i in range(df.shape[0]):
                if df["DAUID"][i] in list_DAUID and df["Att_id"][i] in Sc_List:
                    result2 = result2.append(df.iloc[[i]])
        
            result = pd.concat([result1, result2])
            result = result[["DAUID","Att_id","Total Sex"]].set_index(["DAUID"])
            return result
        else:
            return df2
    else:
        return df2
    #return df2

In [36]:
FindDF(DAUID_list, [51,59,74]) # 51, 59 are primary attributes 

Primary attribute ID is included in your list provided. 
Would you like to see detail table with sub-attributes? [Y/N] Y


,Att_id,Total Sex
DAUID,,
60010189,51,170
60010189,52,65
60010189,53,70
60010189,54,15
60010189,55,20
60010189,56,10
60010189,57,360
60010189,58,2.0
60010189,59,305


In [37]:
FindDF(DAUID_list, [51,59,74])

Primary attribute ID is included in your list provided. 
Would you like to see detail table with sub-attributes? [Y/N] N


,Att_id,Total Sex
DAUID,,
60010189,51,170
60010189,59,305
60010189,74,100
60010195,51,315
60010195,59,530
60010195,74,160


In [38]:
FindDF(DAUID_list, [53,77,81]) 
# 53,77,81 - All are not primary; they are all sub-attributes. 
# Accordingly, no question will be given to user if they want to see detail table, as all are already specific sub-attributes.

,Att_id,Total Sex
DAUID,,
60010189,53,70
60010189,77,25
60010189,81,95
60010195,53,105
60010195,77,45
60010195,81,125
